# Machine Translation with Transformer

In [1]:
import sys
sys.path.append('../..')

In [2]:
import yaml
import time
import glob
import datetime
from dataclasses import asdict
from app.cli.modules.node import get_config_db, get_node_manager
from app.cli.modules.role import get_role_manager
from app.cli.modules.cluster import get_cluster_config_db, get_cluster_manager
from clap.utils import float_time_to_string, path_extend, yaml_load
from clap.executor import SSHCommandExecutor, AnsiblePlaybookExecutor


In [3]:
configuration_db = get_config_db()
cluster_config_db = get_cluster_config_db()
node_manager = get_node_manager()
role_manager = get_role_manager()
cluster_manager = get_cluster_manager()
# Private's path (usually ~/.clap/private/) will be used for other methods
private_path = node_manager.private_path

Redefinition of setup setup-initial. Skipping


`cluster_config_db` will load all cluster configs at `~/.clap/configs/clusters/` and will store all in the `clusters` member. `clusters` is a dictionary, where the keys are the name of cluster configuration the values are dataclasses of type `ClusterConfig`.

Let's list all cluster configurations and get the configuration named `npb-cluster`.

In [4]:
print(list(cluster_config_db.clusters.keys()))

['my-cluster', 'example-cluster']


In [5]:
npb_cluster_config = cluster_config_db.clusters['my-cluster']
print(npb_cluster_config)

ClusterConfig(cluster_config_id='my-cluster', options=None, before_all=[], before=[SetupConfig(roles=[RoleAdd(name='gan', extra={})], actions=[RoleActionType(role='gan', action='update-packages', extra={})])], after_all=[], after=[SetupConfig(roles=[], actions=[RoleActionType(role='gan', action='install-packages', extra={'packages': 'python3-pip, build-essential, cmake, openmpi-bin, openmpi-common, openmpi-doc, libopenmpi-dev'})]), SetupConfig(roles=[], actions=[RoleActionType(role='gan', action='run-command', extra={'cmd': 'sudo apt-get -y install python-is-python3'})]), SetupConfig(roles=[], actions=[RoleActionType(role='gan', action='run-command', extra={'cmd': 'sudo pip install mxnet gluonnlp sacremoses'})]), SetupConfig(roles=[], actions=[RoleActionType(role='gan', action='run-command', extra={'cmd': 'sudo rm -rf optimizer-clap-app/ && git clone https://github.com/robertopossidente/optimizer-clap-app.git'})]), SetupConfig(roles=[], actions=[RoleActionType(role='gan', action='run-c

The configuration is a dataclass, so it can be ful converted to a dict, with `asdict` function.

In [6]:
npb_cluster_config_dict = asdict(npb_cluster_config)
print(yaml.dump(npb_cluster_config_dict, indent=4))

after:
-   actions:
    -   action: install-packages
        extra:
            packages: python3-pip, build-essential, cmake, openmpi-bin, openmpi-common,
                openmpi-doc, libopenmpi-dev
        role: gan
    roles: []
-   actions:
    -   action: run-command
        extra:
            cmd: sudo apt-get -y install python-is-python3
        role: gan
    roles: []
-   actions:
    -   action: run-command
        extra:
            cmd: sudo pip install mxnet gluonnlp sacremoses
        role: gan
    roles: []
-   actions:
    -   action: run-command
        extra:
            cmd: sudo rm -rf optimizer-clap-app/ && git clone https://github.com/robertopossidente/optimizer-clap-app.git
        role: gan
    roles: []
-   actions:
    -   action: run-command
        extra:
            cmd: sudo mkdir -p /etc/ansible/facts.d/ && sudo cp optimizer-clap-app/times.fact
                /etc/ansible/facts.d/ && sudo chown ubuntu:ubuntu /etc/ansible/facts.d/times.fact
               

We can start a cluster, based on a cluster configuration, using the `start_cluster`  method from `ClusterManager` class The function will return a cluster id that will be used for other methods.

In [7]:
cluster_id = cluster_manager.start_cluster(npb_cluster_config)
print(cluster_id)

[WARNING]: provided hosts list is empty, only localhost is available. Note that
the implicit localhost does not match 'all'

PLAY [localhost] ***************************************************************

TASK [Starting 1 type-a instances (timeout 600 seconds)] ***********************
changed: [localhost]

PLAY RECAP *********************************************************************
localhost                  : ok=1    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   

[WARNING]: provided hosts list is empty, only localhost is available. Note that
the implicit localhost does not match 'all'

PLAY [localhost] ***************************************************************

TASK [Tagging instances] *******************************************************
changed: [localhost] => (item={'id': 'i-08b8c63309b27d596', 'name': 'KevinWesolowski-96d7a5bb'})

PLAY RECAP *********************************************************************
localhost              

We can get a full cluster information using `get_cluster_by_id` method from `ClusterManager` class. It will return a dataclass of type `ClusterDescriptor` that has all the information of a custer. TO get all clusters in the repository, `get_all_clusters` function returns a list of `ClusterDescriptor`. 

Let's print the `ClusterDescriptor` from cluster the recently created cluster `cluster-da580f1038254cfa98b203ca109ecb53` in YAML format.

Given a cluster id, we can get all CLAP nodes that belongs to this cluster, using `get_all_cluster_nodes` method from `ClusterManager` class. It wil return a list of node ids, which can be used with several CLAP modules, such as `NodeManager` and `RoleManager` classes..

In [8]:
cluster_nodes = cluster_manager.get_all_cluster_nodes(cluster_id)
print(cluster_nodes)

['96d7a5bb168f4456bb89fdef16762ed9', '5930aef76c1648deb35816a48a552409']


Using the `get_cluster_nodes_types` method from `ClusterManager` class will result in a dictionary where the key are the cluster node types (e.g., `npb-type-b`) and the values are a list of node ids of nodes from that type.

In [9]:
cluster_nodes_with_type = cluster_manager.get_cluster_nodes_types(cluster_id)
print(cluster_nodes_with_type)

{'type-a': ['96d7a5bb168f4456bb89fdef16762ed9'], 'type-b': ['5930aef76c1648deb35816a48a552409']}


In [10]:
cluster_nodes_with_type = cluster_manager.get_cluster_nodes_types(cluster_id)
print(yaml.dump(cluster_nodes_with_type))

type-a:
- 96d7a5bb168f4456bb89fdef16762ed9
type-b:
- 5930aef76c1648deb35816a48a552409



In [11]:
cluster_manager.setup_cluster(cluster_id, nodes_being_added=None, max_workers=1, start_at_stage='before')
#cluster_manager.setup_cluster(cluster_id, nodes_being_added=None, max_workers=1, start_at_stage='after_all')


PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [96d7a5bb168f4456bb89fdef16762ed9]
ok: [5930aef76c1648deb35816a48a552409]

TASK [Perform package list update] *********************************************
changed: [96d7a5bb168f4456bb89fdef16762ed9]
changed: [5930aef76c1648deb35816a48a552409]

PLAY RECAP *********************************************************************
5930aef76c1648deb35816a48a552409 : ok=2    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
96d7a5bb168f4456bb89fdef16762ed9 : ok=2    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   


PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [96d7a5bb168f4456bb89fdef16762ed9]

TASK [Perform package list update] ******************

In [12]:
cluster_nodes_with_type = cluster_manager.get_cluster_nodes_types(cluster_id)
cluster_nodes = cluster_manager.get_all_cluster_nodes(cluster_id)
host_nodes = node_manager.get_nodes_by_id(cluster_nodes)
print(host_nodes)
hosts_ips=''
for node in host_nodes:
    print (f"{node.node_id}: Status: {node.status}; IP: {node.ip}")
    hosts_ips = hosts_ips + node.ip + ':1,'
hosts_ips = hosts_ips[0:len(hosts_ips)-1]
print(hosts_ips)

master_id = cluster_nodes_with_type['type-a']
print(master_id)
master_node = node_manager.get_nodes_by_id(master_id)
print(master_node)

[NodeDescriptor(node_id='96d7a5bb168f4456bb89fdef16762ed9', configuration=InstanceInfo(provider=ProviderConfigAWS(provider_config_id='aws-config-us-east-1', region='us-east-1', access_keyfile='aws-keypair.pub', secret_access_keyfile='aws-keypair.pem', vpc=None, url='https://ec2.us-east-1.amazonaws.com', provider='aws'), login=LoginConfig(login_config_id='login-ubuntu', user='ubuntu', keypair_name='gan-clap-keypair', keypair_public_file='gan-clap-keypair.pub', keypair_private_file='gan-clap-keypair.pem', ssh_port=22, sudo=True, sudo_user='root'), instance=InstanceConfigAWS(instance_config_id='type-a', provider='aws-config-us-east-1', login='login-ubuntu', flavor='t2.medium', image_id='ami-09e67e426f25ce0d7', security_group='gan-clap-sg-allopen', boot_disk_size=16, boot_disk_device=None, boot_disk_type=None, boot_disk_iops=None, boot_disk_snapshot=None, placement_group=None, price=None, timeout=None, network_ids=[])), nickname='KevinWesolowski', ip='34.224.84.67', type='cloud', cloud_ins

In [13]:
command_to_execute = """
horovodrun -np 2 -H {} python /home/ubuntu/optimizer-clap-app/machine-translation/my-train.py """.format(hosts_ips)
print(command_to_execute)


horovodrun -np 2 -H 34.224.84.67:1,3.236.92.158:1 python /home/ubuntu/optimizer-clap-app/machine-translation/my-train.py 


In [14]:
'''command_to_execute = """
mpirun -np 1 -H localhost:1 -bind-to none -map-by slot python /home/ubuntu/optimizer-clap-app/machine-translation/my-train.py 2>&1 > log.txt 
echo Launch Machine Translation by ssh
"""
print(command_to_execute)'''
#command_to_execute = """horovodrun"""
executor = SSHCommandExecutor(command_to_execute, master_node, private_path)
result = executor.run()

for node_id, res in result.items():
    print(f"Node id {node_id}, executed the command: {res.ok}, ret code: {res.ret_code}")
    #resut is a dataclass, we can convert to a dictionary
    res_dict = asdict(res)
    print('-----')
    # Dump dictionary in YAML format
    print(yaml.dump(res_dict, indent=4, sort_keys=True))

Node id 96d7a5bb168f4456bb89fdef16762ed9, executed the command: True, ret code: 0
-----
error: null
ok: true
ret_code: 0
stderr_lines:
- '[1,1]<stderr>:INFO:root:Namespace(batch_size=64, dtype=''float32'', epochs=5, lr=0.01,
    momentum=0.9, no_cuda=True)

    '
- '[1,0]<stderr>:INFO:root:Namespace(batch_size=64, dtype=''float32'', epochs=5, lr=0.01,
    momentum=0.9, no_cuda=True)

    '
stdout_lines:
- '[1,0]<stdout>:Loading dataset...

    '
- '[1,1]<stdout>:Loading dataset...

    '
- '[1,0]<stdout>:Loading dataset...

    '
- '[1,1]<stdout>:Loading dataset...

    '
- '[1,1]<stdout>:Loading dataset...

    '
- '[1,0]<stdout>:Loading dataset...

    '
- '[1,1]<stdout>:Numero de Epochs = 1

    '
- '[1,1]<stdout>:Epoch 0 - train_one_epoch started

    '
- '[1,1]<stdout>:[MO833] Rank,1,Initialization Time: 0.000056

    '
- '[1,0]<stdout>:Numero de Epochs = 1

    '
- '[1,0]<stdout>:Epoch 0 - train_one_epoch started

    '
- '[1,0]<stdout>:[MO833] Rank,0,Initialization Time: 0.00003

In [15]:
cat /home/ubuntu/.clap/roles/roles/getfacts.yml

---
- hosts: all
  gather_facts: True    # Query a set of variables in remote hosts
  gather_subset: min
  tasks:
  - name: Get iteration time from a fact
    set_fact:
      iteration_time: "{{ ansible_local.times.iteration_time }}"

In [16]:
nodes = node_manager.get_nodes_by_id(cluster_nodes)
for node in nodes:
    print (f"{node.node_id}: Status: {node.status}; IP: {node.ip}, Type: {node.type}")

playbook_file = path_extend('/home/ubuntu/.clap/roles/roles/getfacts.yml')
inventory = AnsiblePlaybookExecutor.create_inventory(nodes, private_path)
executor = AnsiblePlaybookExecutor(playbook_file, private_path, inventory=inventory)

result = executor.run()

print(f"Did the playbook executed? {result.ok}")
print(f"Ansible playbook return code: {result.ret_code}")
print(f"Let's check how nodes executed: ")
for node_id, status in result.hosts.items():
    print(f"    Node {node_id}: {status}")
print(f"Let's check variables set using set_fact module: ")
times={}
for node_id, facts in result.vars.items():
    print(f"    Node {node_id}: {facts}")
    times[node_id] = facts['iteration_time']
print('%s' % str(times))
# Dump dictionary in YAML format
print(yaml.dump(times, indent=4, sort_keys=True))

pi_logs_dir = '/home/ubuntu/CLAP/'
print('%s' % str(pi_logs_dir))
#timestamp_dir = path_extend(pi_logs_dir, '/timestamp/')
timestamp_dir = pi_logs_dir + 'timestamp/'
print('%s' % str(timestamp_dir))
os.makedirs(timestamp_dir, exist_ok=True)
timestamp_file = timestamp_dir + str(int(time.time())) +  '.out'
print('%s' % str(timestamp_file))
with open(timestamp_file, 'a') as outfile:
   yaml.dump(times, outfile, default_flow_style=False)


96d7a5bb168f4456bb89fdef16762ed9: Status: reachable; IP: 34.224.84.67, Type: cloud
5930aef76c1648deb35816a48a552409: Status: reachable; IP: 3.236.92.158, Type: cloud

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
[DEPRECATION WARNING]: Distribution ubuntu 20.04 on host 
96d7a5bb168f4456bb89fdef16762ed9 should use /usr/bin/python3, but is using 
/usr/bin/python for backward compatibility with prior Ansible releases. A 
future Ansible release will default to using the discovered platform python for
 this host. See https://docs.ansible.com/ansible/2.11/reference_appendices/inte
rpreter_discovery.html for more information. This feature will be removed in 
version 2.12. Deprecation warnings can be disabled by setting 
deprecation_warnings=False in ansible.cfg.
ok: [96d7a5bb168f4456bb89fdef16762ed9]
[DEPRECATION WARNING]: Distribution ubuntu 20.04 on host 
5930aef76c1648deb358

In [17]:
root_dir='/home/ubuntu/CLAP/'
experiment_id = 'Transformer' 
vm_price_file = '/home/ubuntu/CLAP/vm_prices.yaml'
report_time = 60

'''nodes = node_manager.get_nodes_by_id(cluster_nodes)
for node in nodes:
    print (f"{node.node_id}: Status: {node.status}; IP: {node.ip}, Type: {node.type}")'''

instance_costs = yaml_load(vm_price_file)
node_prices={}

for node in nodes:
    print('---------')
    print(f"Node Id: {node.node_id}, created at {float_time_to_string(node.creation_time)}; Status: {node.status}")
    print('---------')
    # Or can be converted to a dict
    node_dict = asdict(node)
    # Printing dict in YAML format
    #print(yaml.dump(node_dict, indent=4))
    #print('**********')
    instance_flavor = node_dict['configuration']['instance']['flavor']
    node_prices[node.node_id] = float(times[node.node_id]) * float(instance_costs[instance_flavor])
    print(f"Instance Flavor: {instance_flavor}, Instance Cost: {instance_costs[instance_flavor]}, Iteration Time: {times[node.node_id]}, Node Price: {node_prices[node.node_id]}")
    print('---------')
print('Node Prices')
print(str(node_prices))


---------
Node Id: 96d7a5bb168f4456bb89fdef16762ed9, created at 08-07-21 20:12:34; Status: reachable
---------
Instance Flavor: t2.medium, Instance Cost: 0.0464, Iteration Time: 1.7858874797821045, Node Price: 0.08286517906188964
---------
---------
Node Id: 5930aef76c1648deb35816a48a552409, created at 08-07-21 20:13:04; Status: reachable
---------
Instance Flavor: t2.large, Instance Cost: 0.0555, Iteration Time: 1.6631438732147217, Node Price: 0.09230448496341706
---------
Node Prices
{'96d7a5bb168f4456bb89fdef16762ed9': 0.08286517906188964, '5930aef76c1648deb35816a48a552409': 0.09230448496341706}


In [18]:
import app.cli.modules.optimizer as opt
is_otimizado = opt.optimize_it(cluster_id, experiment_id, vm_price_file,root_dir, report_time)

'''metrics = node_prices

cluster = cluster_manager.get_cluster_by_id(cluster_id)
cluster_dict = asdict(cluster)
print('cluster -> yaml.dump \n')
print(yaml.dump(cluster_dict, indent=4))
cluster_nodes = cluster_manager.get_all_cluster_nodes(cluster_id)
print('cluster_nodes \n')
print(cluster_nodes)
cluster_nodes_with_type = cluster_manager.get_cluster_nodes_types(cluster_id)
print('cluster_nodes_with_type \n')
print(cluster_nodes_with_type)
print('cluster_nodes_with_type -> yaml.dump \n')
print(yaml.dump(cluster_nodes_with_type))

nodes = node_manager.get_nodes_by_id(cluster_nodes)

higher_price = 0
lower_price = 1000
higher_price_node_id=[]
lower_price_node_id=[]
for node in nodes:
    node_dict = asdict(node)
    if(float(metrics[node.node_id]) > float(higher_price)):
        higher_price = float(metrics[node.node_id])
        higher_price_node_id.clear()
        higher_price_node_id.append(node.node_id)
        high_instance_type = node_dict['configuration']['instance']['instance_config_id']
        high_instance_flavor = node_dict['configuration']['instance']['flavor']
    if(float(metrics[node.node_id]) < float(lower_price)):
        lower_price = float(metrics[node.node_id])
        lower_price_node_id.clear()
        lower_price_node_id.append(node.node_id)
        low_instance_type = node_dict['configuration']['instance']['instance_config_id']
        low_instance_flavor = node_dict['configuration']['instance']['flavor']

print(f"High Price Node ID: {higher_price_node_id}, Instance Type: {high_instance_type}") 
print(f"Lower Price Node ID: {lower_price_node_id}, Instance Type: {low_instance_type}")

new_node_id = cluster_manager.grow(cluster_id, node_type=low_instance_type, count=1, min_count=1)
print(f"New Node: {new_node_id}")

#if(high_instance_flavor!=low_instance_flavor)
alive_node = node_manager.is_alive(new_node_id)
for node_id, alive_flag in alive_node.items():
    if(alive_flag == True):
        print(f"[INIT] New Node: {new_node_id}")
        new_node_ids = node_manager.get_nodes_by_id(new_node_id)
        for node in new_node_ids:
            print(f"[INIT] {float_time_to_string(node.creation_time)}: Created Node:{node.node_id} of #type {low_instance_flavor}")
        print(str(new_node_ids))
        #new_nodes_types = {
        #   low_instance_type: new_node_id
        #}
        #cluster_manager.setup_cluster(cluster_id, nodes_being_added=new_nodes_types, 
        #                                                  start_at_stage='before_all')
        stopped_node_id = node_manager.stop_nodes(higher_price_node_id)
        print(f"[STOP] %s: Terminated Node:{higher_price_node_id}" % (datetime.datetime.now()))
        result = True
    else:
        result = False'''






32mok: [dbd8e0a0b2ba471fba6937d2b8a3a188]

TASK [Perform package list update] *********************************************
changed: [dbd8e0a0b2ba471fba6937d2b8a3a188]

PLAY RECAP *********************************************************************
dbd8e0a0b2ba471fba6937d2b8a3a188 : ok=2    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   


PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [dbd8e0a0b2ba471fba6937d2b8a3a188]

TASK [Perform package list update] *********************************************
changed: [dbd8e0a0b2ba471fba6937d2b8a3a188]

PLAY RECAP *********************************************************************
dbd8e0a0b2ba471fba6937d2b8a3a188 : ok=2    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   


PLAY [all] *********************************************************************

TASK [

'metrics = node_prices\n\ncluster = cluster_manager.get_cluster_by_id(cluster_id)\ncluster_dict = asdict(cluster)\nprint(\'cluster -> yaml.dump \n\')\nprint(yaml.dump(cluster_dict, indent=4))\ncluster_nodes = cluster_manager.get_all_cluster_nodes(cluster_id)\nprint(\'cluster_nodes \n\')\nprint(cluster_nodes)\ncluster_nodes_with_type = cluster_manager.get_cluster_nodes_types(cluster_id)\nprint(\'cluster_nodes_with_type \n\')\nprint(cluster_nodes_with_type)\nprint(\'cluster_nodes_with_type -> yaml.dump \n\')\nprint(yaml.dump(cluster_nodes_with_type))\n\nnodes = node_manager.get_nodes_by_id(cluster_nodes)\n\nhigher_price = 0\nlower_price = 1000\nhigher_price_node_id=[]\nlower_price_node_id=[]\nfor node in nodes:\n    node_dict = asdict(node)\n    if(float(metrics[node.node_id]) > float(higher_price)):\n        higher_price = float(metrics[node.node_id])\n        higher_price_node_id.clear()\n        higher_price_node_id.append(node.node_id)\n        high_instance_type = node_dict[\'config

## Stopping cluster

Finally we can stop the cluster (and stop all nodes) using the `stop_cluster` command. This will also remove the cluster from cluster repository.

Other similar functions are:
* `resume_cluster`: That will resume all paused nodes of a cluster  
* `pause_cluster`: That will pause all nodes of a cluster
* `is_alive`: That will check if all cluster nodes are alive

In [19]:
#cluster_manager.stop_cluster(cluster_id)

In [20]:
'''clusters = cluster_manager.get_all_clusters()
for cluster in clusters:
    cluster_manager.stop_cluster(cluster)'''


'clusters = cluster_manager.get_all_clusters()\nfor cluster in clusters:\n    cluster_manager.stop_cluster(cluster)'

In [21]:
#node_manager.stop_nodes(['2c4610c8c02b41f983d96fcf928e0a28'])
